In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *


/Users/sam11/Git/ml-py


### Greystone

In [2]:
user = "neo4j"
password = 'GREYSTONE!!'

csv_file = os.path.join(project_path ,'data', 'gs_data', 'READABLE_TAGS.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'gs_data', 'gs_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'gs_data', 'BINARY_TAGS.csv')
binNcsv_file = os.path.join(project_path ,'data', 'gs_data', 'BINARY_RELATIONS.csv')


### Mine

In [ ]:
user = "neo4j"
password = 'MINE!!'

csv_file = os.path.join(project_path ,'data', 'public', 'READABLE_TAGS.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'public', 'mine_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'public', 'BINARY_TAGS.csv')
binNcsv_file = os.path.join(project_path ,'data', 'public', 'BINARY_RELATIONS.csv')


### HVAC

In [ ]:
user = "neo4j"
password = 'HVAC!!'

csv_file = os.path.join(project_path ,'data', 'hvac_data', 'hvac_raw.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'hvac_data', 'hvac_header.yaml')

### Others

In [ ]:
user = getpass.getuser()
password = getpass.getpass("Password:")

csv_file = os.path.join(project_path ,'data', 'hvac_data', 'hvac_raw.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'hvac_data', 'hvac_header.yaml')

## Connect to DB

In [3]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

yaml file open


## Load CSV

In [4]:
dataframe = pd.read_csv(csv_file, index_col=0, header=0)
dataframe.fillna("", inplace=True)


dataframe_bincsv = pd.read_csv(bincsv_file, index_col=0, header=[0,1])
dataframe_binNcsv = pd.read_csv(binNcsv_file, index_col=0, header=[0,1])

csvSchemas_dict = openYAMLFile(csvSchemas_file)

yaml file open


### Clean the database

In [5]:
database.deleteData()

database.dropConstraints()
database.dropIndexes()

database.createIndexes()
database.createConstraints()

CREATE CONSTRAINT ON (issue:ISSUE) ASSERT issue.id IS UNIQUE


1

## Store the data

In [6]:
#done = graphDatabase_from_TaggedCSV(database, dataframe, csvSchemas_dict)
done = graphDatabase_from_binnaryCSV(database, dataframe, dataframe_bincsv, dataframe_binNcsv, csvSchemas_dict)

  0%|          | 0/813 [00:00<?, ?it/s]

-- Queries from the HISTORICAL dataset created !!


  5%|▌         | 53/1024 [00:00<00:01, 529.09it/s]

-- Queries from the 1GRAM TAGGED dataset created !!


  0%|          | 0/5283 [00:00<?, ?it/s]

-- Queries from the NGRAM TAGGED dataset created !!

MATCH (solutionItem:TAG:N_GRAM:PROBLEM_ITEM)
WITH solutionItem, split(solutionItem.keyword, " ") AS halfTags
UNWIND halfTags AS halfTag
MATCH (unkknown:TAG:ONE_GRAM:UNKNOWN{keyword: halfTag})
MERGE (solutionItem)-[:COMPOSED_OF]->(unknown)
-- Queries to linked 1GRAM TAG and NGRAM TAG created!!
-- Queries to update ITEM TAG to ISSUE relationship created!!


100%|██████████| 5283/5283 [02:34<00:00, 34.11it/s]

-- Queries stored into the database


# KPI

## Create the object for the KPI

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *

databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)

In [ ]:
m = MachineKpi(operator="=", name = "h1", databaseInfo=databaseSchema_dict)
t = TechnicianKpi(operator=".-", name="steve", result="name", databaseInfo=databaseSchema_dict)
i = TagItemKpi(operator="<>", keyword="valve", result="keyword", databaseInfo=databaseSchema_dict)

In [ ]:
kpi = m+t+i
print(kpi.cypher_createQuery())

In [ ]:
done, filterResult = database.runQuery(kpi.cypher_createQuery())

dataframe = resultToObservationDataframe(filterResult)
dataframe

----------------------------

In [ ]:
h1 = dataframe_bincsv.columns

In [ ]:
h2 = dataframe_binNcsv.columns